<img src="https://github.com/nicholasmetherall/digital-earth-pacific-macblue-activities/blob/main/attachments/images/DE_Pacific_banner.JPG?raw=true" width="900"/>

Figure 1.1.a. Jupyter environment + Python notebooks

# Digital Earth Pacific Notebook 1 prepare postcard and load data to csv

The objective of this notebook is to prepare a geomad postcard for your AOI (masking, scaling and loading additional band ratios and spectral indices) and sampling all the datasets into a csv based on your training data geodataframe.

In [1]:
# # This cell is for papermill parameters. DO NOT CHANGE THE VARIABLE NAMES.
# # Default values for manual execution (papermill will override these)
# input_geojson_path = None
# output_csv_path = None

## Step 1.1: Configure the environment

In [2]:
import os
from datetime import datetime
from shapely.geometry import Polygon
from shapely import box
from pyproj import CRS 
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio as rio
import xarray as xr
import rioxarray
from ipyleaflet import basemaps
from numpy.lib.stride_tricks import sliding_window_view
import pystac_client
from dask.distributed import Client as DaskClient
from odc.stac import load, configure_s3_access
import planetary_computer
from odc.stac import load
from pystac.client import Client
from skimage.feature import graycomatrix, graycoprops
from utils import load_data, scale, calculate_band_indices, apply_mask, mask_land, mask_deeps, mask_elevation, all_masks, glcm_features, do_prediction

In [3]:
# Reload scripts and imports
%load_ext autoreload
%autoreload 2

In [4]:
# Predefined variable for title and version

# Enter your initials
initials = "nm"

# Enter your site name
site = "suva_central_2"

# Date
date = datetime.now()

# Make a clean version string
version = f"{initials}-{site}-{date.strftime('%d%m%Y')}"
print(version)

nm-suva_central_2-27072025


In [5]:
gdfs = []
postcards_path = "training-data/postcards"
file_extension: str = ".geojson"

for filename in os.listdir(postcards_path):
    file_path = os.path.join(postcards_path, filename)
    if os.path.isfile(file_path) and filename.endswith(file_extension):
    # try:
        gdf = gpd.read_file(file_path)
        gdfs.append(gdf)

In [6]:
for filename in os.listdir(postcards_path):
    file_path = os.path.join(postcards_path, filename)
    if os.path.isfile(file_path) and filename.endswith(file_extension):
        print(filename) # This line will print the name of each GeoJSON file
        # The rest of your code to read the file and append to gdfs
        # gdf = gpd.read_file(file_path)
        # gdfs.append(gdf)

print("\nFinished listing GeoJSON files.")

nm-efate-27072025_postcard.geojson
suva_west_postcard.geojson
paunganisu_postcard.geojson
nm-suva_central_2-27072025_postcard.geojson
suva_central_2_postcard.geojson
mana-north_postcard.geojson
tulagi_postcard.geojson
serua_2_postcard.geojson
suva_central_postcard.geojson
efate_postcard.geojson
suva_central_1_postcard.geojson
suva_east_postcard.geojson
bootless-bay_postcard.geojson
serua_postcard.geojson
korotogo_postcard.geojson
nm-suva_central_1-27072025_postcard.geojson
yanuca_postcard.geojson
mana-south_postcard.geojson

Finished listing GeoJSON files.


## Step 1.2: Configure STAC access and search parameters

In [7]:
catalog = "https://stac.digitalearthpacific.org"
client = Client.open(catalog)

In [8]:
# filename = "nm-efate-27072025_postcard.geojson"

In [9]:
training = gpd.read_file(f"training-data/postcards/{site}_postcard.geojson")
training = training.to_crs("EPSG:4326")
min_lon, min_lat, max_lon, max_lat = training.total_bounds

bbox = [min_lon, min_lat, max_lon, max_lat]

In [10]:
datetime = "2024"

items = client.search(
    collections=["dep_s2_geomad"],
    datetime=datetime,
    bbox=bbox
).item_collection()

print(f"Found {len(items)} items in for {datetime}")

Found 1 items in for 2024


In [11]:
measurements = ["nir", "red", "blue", "green", "emad", "smad", "bcmad", "green", "nir08", "nir09", "swir16", "swir22", "coastal", "rededge1", "rededge2", "rededge3"]
data = load_data(
    items,
    measurements,
    bbox,
)
    
# Now you can use the 'data' variable
print(data)

<xarray.Dataset> Size: 7MB
Dimensions:      (y: 262, x: 696, time: 1)
Coordinates:
  * y            (y) float64 2kB -2.044e+06 -2.044e+06 ... -2.047e+06 -2.047e+06
  * x            (x) float64 6kB 3.163e+06 3.163e+06 ... 3.17e+06 3.17e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/15)
    nir          (time, y, x) uint16 365kB dask.array<chunksize=(1, 262, 696), meta=np.ndarray>
    red          (time, y, x) uint16 365kB dask.array<chunksize=(1, 262, 696), meta=np.ndarray>
    blue         (time, y, x) uint16 365kB dask.array<chunksize=(1, 262, 696), meta=np.ndarray>
    green        (time, y, x) uint16 365kB dask.array<chunksize=(1, 262, 696), meta=np.ndarray>
    emad         (time, y, x) float32 729kB dask.array<chunksize=(1, 262, 696), meta=np.ndarray>
    smad         (time, y, x) float32 729kB dask.array<chunksize=(1, 262, 696), meta=np.ndarray>
    ...           ...
    swir16       (time, y, x) uint16 365kB dask.array

In [12]:
dask_client = DaskClient(n_workers=1, threads_per_worker=16, memory_limit='16GB')
configure_s3_access(cloud_defaults=True, requester_pays=True)

In [13]:
scaled = scale(data)
scaled = scaled.compute().squeeze()

In [14]:
# Explore the site we are working on
scaled.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [15]:
scaled

<xarray.Dataset> Size: 20MB
Dimensions:      (y: 262, x: 696)
Coordinates:
  * y            (y) float64 2kB -2.044e+06 -2.044e+06 ... -2.047e+06 -2.047e+06
  * x            (x) float64 6kB 3.163e+06 3.163e+06 ... 3.17e+06 3.17e+06
    spatial_ref  int32 4B 3832
    time         datetime64[ns] 8B 2024-01-01
Data variables: (12/15)
    nir          (y, x) float64 1MB 0.0293 0.0295 0.0293 ... 0.0277 0.028 0.0268
    red          (y, x) float64 1MB 0.0527 0.0515 0.0511 ... 0.0312 0.0309
    blue         (y, x) float64 1MB 0.0777 0.0756 0.0757 ... 0.0563 0.0547
    green        (y, x) float64 1MB 0.0874 0.0845 0.0847 ... 0.0419 0.0422 0.041
    emad         (y, x) float32 729kB 0.09404 0.09484 ... 0.09052 0.08485
    smad         (y, x) float32 729kB 4.077e-06 4.962e-06 ... 4.457e-06
    ...           ...
    swir16       (y, x) float64 1MB 0.0253 0.0254 0.0255 ... 0.0234 0.0229
    swir22       (y, x) float64 1MB 0.0236 0.0241 0.0242 ... 0.0224 0.0221
    coastal      (y, x) float64 1MB 0.0763 0.0772 0.0774 ... 0.0626 0.0621
    rededge1     (y, x) float64 1MB 0.0571 0.056 0.0561 ... 0.0303 0.0323 0.0315
    rededge2     (y, x) float64 1MB 0.0312 0.0314 0.0315 ... 0.0293 0.0286
    rededge3     (y, x) float64 1MB 0.0317 0.0327 0.0329 ... 0.0291 0.0284

In [16]:
scaled = calculate_band_indices(scaled)
Dataset = scaled

### GLCM texture analysis

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [17]:
WINDOW_SIZE = 9
LEVELS = 32

# Input
max = scaled.blue.max().values
min = scaled.blue.min().values
# Scale to 0-LEVELS for GLCM
img = ((scaled.blue - min) / (max - min) * (LEVELS - 1)).clip(0, LEVELS - 1).values.astype(np.uint8)

# Extract overlapping windows
patches = sliding_window_view(img, (WINDOW_SIZE, WINDOW_SIZE))
# Shape: (rows, cols, win_y, win_x)


In [18]:
import numpy as np # Ensure numpy is imported if not already

# Assuming 'patches' is a 4D NumPy array with dimensions (y_coords, x_coords, window_y_size, window_x_size)
# To get the first patch (at y=0, x=0), you would index it like this:
sample_patch_data = patches[0, 0, :, :]

# Verify the shape of the extracted sample patch data
print(f"Shape of sample_patch_data: {sample_patch_data.shape}")

# Call glcm_features directly on this 2D sample data
sample_result = glcm_features(sample_patch_data)

# Print the shape of the result to get the number of features
print(f"Shape of glcm_features output for a single patch: {sample_result.shape}")

Shape of sample_patch_data: (9, 9)
Shape of glcm_features output for a single patch: (7,)


In [19]:
# Use apply_ufunc to vectorize over (row, col) dimensions
result = xr.apply_ufunc(
    glcm_features,
    xr.DataArray(patches, dims=["y", "x", "win_y", "win_x"]),
    input_core_dims=[["win_y", "win_x"]],
    output_core_dims=[["feature"]],
    vectorize=True,
    dask="parallelized",
    output_dtypes=[np.float32]
)

# Add coordinates & names
pad = WINDOW_SIZE - 1
result = result.assign_coords({
    "y": scaled.y[: -pad],
    "x": scaled.x[: -pad],
    "feature": ["contrast", "homogeneity", "energy", "ASM", "correlation", "mean", "entropy"]
})

result_bands = result.to_dataset(dim="feature")

# Combine with original
combined = scaled.copy()
combined = combined.assign(result_bands)

combined

<xarray.Dataset> Size: 42MB
Dimensions:      (y: 262, x: 696)
Coordinates:
  * y            (y) float64 2kB -2.044e+06 -2.044e+06 ... -2.047e+06 -2.047e+06
  * x            (x) float64 6kB 3.163e+06 3.163e+06 ... 3.17e+06 3.17e+06
    spatial_ref  int32 4B 3832
    time         datetime64[ns] 8B 2024-01-01
Data variables: (12/34)
    nir          (y, x) float64 1MB 0.0293 0.0295 0.0293 ... 0.0277 0.028 0.0268
    red          (y, x) float64 1MB 0.0527 0.0515 0.0511 ... 0.0312 0.0309
    blue         (y, x) float64 1MB 0.0777 0.0756 0.0757 ... 0.0563 0.0547
    green        (y, x) float64 1MB 0.0874 0.0845 0.0847 ... 0.0419 0.0422 0.041
    emad         (y, x) float32 729kB 0.09404 0.09484 ... 0.09052 0.08485
    smad         (y, x) float32 729kB 4.077e-06 4.962e-06 ... 4.457e-06
    ...           ...
    homogeneity  (y, x) float32 729kB 0.9167 0.9028 0.8889 0.875 ... nan nan nan
    energy       (y, x) float32 729kB 0.7074 0.6844 0.6623 ... nan nan nan
    ASM          (y, x) float32 729kB 0.5004 0.4684 0.4387 ... nan nan nan
    correlation  (y, x) float32 729kB 0.538 0.4815 0.428 0.3769 ... nan nan nan
    mean         (y, x) float32 729kB 2.764 2.75 2.736 2.722 ... nan nan nan nan
    entropy      (y, x) float32 729kB 1.389 1.47 1.542 1.608 ... nan nan nan nan

In [20]:
masked_combined, mask = all_masks(combined, return_mask = True)
mask.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [21]:
masked_combined.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

/srv/conda/envs/notebook/lib/python3.10/site-packages/odc/geo/_rgba.py:55: RuntimeWarning: invalid value encountered in cast
  return x.astype("uint8")


### Postcard csv

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [22]:
# Reproject training data to the GeoMAD CRS and convert to xarray
training_reprojected = training.to_crs(masked_combined.odc.crs)
training_da = training_reprojected.assign(
    x=training_reprojected.geometry.x, y=training_reprojected.geometry.y
).to_xarray()

# Extract training values from the masked dataset
training_values = (
    masked_combined.sel(training_da[["x", "y"]], method="nearest")
    .squeeze()
    .compute()
    .to_pandas()
)
training_values

,nir,red,blue,green,emad,smad,bcmad,nir08,nir09,swir16,...,homogeneity,energy,ASM,correlation,mean,entropy,y,x,spatial_ref,time
index,,,,,,,,,,,,,,,,,,,,,
0,0.0353,0.0512,0.0650,0.0730,0.076013,0.000003,0.000020,0.0334,0.0451,0.0252,...,0.923611,0.609452,0.371431,0.693913,1.479167,1.615270,-2044875.0,3169615.0,3832,2024-01-01
1,0.0375,0.0547,0.0663,0.0744,0.075037,0.000003,0.000020,0.0333,0.0445,0.0246,...,0.923611,0.618251,0.382234,0.687142,1.423611,1.596766,-2044855.0,3169615.0,3832,2024-01-01
2,0.0486,0.0527,0.0612,0.0683,0.090950,0.000004,0.000024,0.0438,0.0433,0.0225,...,0.930556,0.621751,0.386574,0.718750,1.444444,1.570951,-2044775.0,3169715.0,3832,2024-01-01
3,0.0418,0.0496,0.0589,0.0656,0.085275,0.000004,0.000024,0.0416,0.0436,0.0232,...,0.937500,0.652260,0.425444,0.731288,1.368056,1.485252,-2044755.0,3169725.0,3832,2024-01-01
4,0.0363,0.0419,0.0535,0.0604,0.090661,0.000007,0.000029,0.0310,0.0465,0.0197,...,0.930556,0.710916,0.505401,0.629630,0.750000,1.358178,-2045205.0,3163945.0,3832,2024-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,0.0299,0.0497,0.0659,0.0789,0.078843,0.000006,0.000023,0.0259,0.0396,0.0175,...,0.902778,0.764520,0.584491,0.248882,1.847222,1.196774,-2045265.0,3164895.0,3832,2024-01-01
517,0.0289,0.0483,0.0654,0.0764,0.067530,0.000006,0.000021,0.0256,0.0393,0.0167,...,0.819444,0.510310,0.260417,0.349548,1.513889,2.084013,-2045445.0,3164865.0,3832,2024-01-01
518,0.0335,0.0497,0.0664,0.0765,0.079660,0.000005,0.000023,0.0286,0.0412,0.0198,...,0.838889,0.617548,0.381366,0.295597,2.291667,1.975978,-2045455.0,3164775.0,3832,2024-01-01


In [23]:
# Join the training data with the extracted values and remove unnecessary columns
training_array = pd.concat([training["cc_id"], training_values], axis=1)

# Drop rows where there was no data available
training_array = training_array.dropna()

# Preview our resulting training array
training_array.head()

,cc_id,nir,red,blue,green,emad,smad,bcmad,nir08,nir09,...,homogeneity,energy,ASM,correlation,mean,entropy,y,x,spatial_ref,time
0,6,0.0353,0.0512,0.0650,0.0730,0.076013,0.000003,0.000020,0.0334,0.0451,...,0.923611,0.609452,0.371431,0.693913,1.479167,1.615270,-2044875.0,3169615.0,3832,2024-01-01
1,6,0.0375,0.0547,0.0663,0.0744,0.075037,0.000003,0.000020,0.0333,0.0445,...,0.923611,0.618251,0.382234,0.687142,1.423611,1.596766,-2044855.0,3169615.0,3832,2024-01-01
2,11,0.0486,0.0527,0.0612,0.0683,0.090950,0.000004,0.000024,0.0438,0.0433,...,0.930556,0.621751,0.386574,0.718750,1.444444,1.570951,-2044775.0,3169715.0,3832,2024-01-01
3,11,0.0418,0.0496,0.0589,0.0656,0.085275,0.000004,0.000024,0.0416,0.0436,...,0.937500,0.652260,0.425444,0.731288,1.368056,1.485252,-2044755.0,3169725.0,3832,2024-01-01
4,11,0.0363,0.0419,0.0535,0.0604,0.090661,0.000007,0.000029,0.0310,0.0465,...,0.930556,0.710916,0.505401,0.629630,0.750000,1.358178,-2045205.0,3163945.0,3832,2024-01-01


In [24]:
print(training_array.shape[1], 'total columns')
print('columns included', training_array.columns)

39 total columns
columns included Index(['cc_id', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1', 'rededge2',
       'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi', 'ndwi',
       'b_g', 'b_r', 'mci', 'ndci', 'ln_bg', 'contrast', 'homogeneity',
       'energy', 'ASM', 'correlation', 'mean', 'entropy', 'y', 'x',
       'spatial_ref', 'time'],
      dtype='object')


In [25]:
standard_schema = ['cc_id', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1',
       'rededge2', 'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi',
       'ndwi', 'b_g', 'b_r', 'mci', 'ndci', 'ln_bg', 'contrast', 'homogeneity',
       'energy', 'ASM', 'correlation', 'mean', 'entropy', 'y', 'x', 'time',
       'spatial_ref']

In [26]:
training_array=training_array[standard_schema]

In [27]:
training_array=training_array.drop(columns=["spatial_ref", "time"])

In [28]:
# Write the training data to a CSV file
training_array.to_csv(f"training-data/csvs/{version}-training.csv", index=False)

In [29]:
training_array["cc_id"].dtype

dtype('int32')